### Processing user-game data

In [1]:
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [2]:
df_ui = parse('australian_users_items.json.gz')

In [3]:
ui_dict = {}

In [4]:
for d in df_ui:
    ui_dict[int(d['steam_id'])] = [{'game_id': int(item['item_id']), 'play_time': item['playtime_forever']} for item in d['items']]

In [5]:
import json

with open('ui_dict.json', 'w') as js:
    json.dump(ui_dict, js)

### Processing game data

In [7]:
df_games = parse('steam_games.json.gz')

In [8]:
tags = []
game_id_dict = {}
for d in df_games:
    if 'id' not in d:
        continue
    game_id_dict[int(d['id'])] = []

    if 'genres' not in d.keys():
        continue
    for genre in d['genres']:
        if genre not in tags:
            tags.append(genre)

    if 'specs' not in d.keys():
        continue
    for spec in d['specs']:
        if spec not in tags:
            tags.append(spec)

In [9]:
"""Turn game id to 1-hot vectors"""

df_games = parse('steam_games.json.gz')
num_tags = len(tags)
for d in df_games:
    if 'id' not in d:
        continue
    game_id = int(d['id'])

    vec_1hot = [0]*num_tags
    if 'genres' not in d.keys():
        continue
    for genre in d['genres']:
        vec_1hot[tags.index(genre)] = 1

    if 'specs' not in d.keys():
        continue
    for spec in d['specs']:
        vec_1hot[tags.index(spec)] = 1

    game_id_dict[game_id] = vec_1hot


In [10]:
with open('game_tags.json', 'w') as js:
    json.dump({'tags': tags}, js)

In [10]:
with open('game_dict.json', 'w') as js:
    json.dump(game_id_dict, js)

### Customer Embedding

In [ ]:
import numpy as np

## {user_id: history \in (53, T)}
for user in ui_dict:
    ui_dict[user]['history'] = np.array([
        game_id_dict[game_id] for game_id in ui_dict[user]['game_id']
    ])
    
